# Day 14
https://adventofcode.com/2017/day/14

In [1]:
import aocd
data = aocd.get_data(year=2017, day=14)

In [2]:
from dataclasses import dataclass
from functools import reduce
from math import prod
from operator import __xor__

#### Adapted code from Day 10 Knot Hash algorithm

In [3]:
def read_lengths(text):
    return tuple(ord(char) for char in text)

In [4]:
def slice_circle(circle, start, length):
    return circle[start:start+length] + circle[0:max(start+length-len(circle), 0)]

In [5]:
def reverse_section(circle, pos, length):
    section = slice_circle(circle, pos, length)
    reversed_section = section[::-1]
    replacements = dict(zip(section, reversed_section))
    return tuple(replacements.get(item, item) for item in circle)

In [6]:
def tie_knots(size, lengths):
    circle = tuple(range(size))
    pos = 0
    skip = 0
    for length in lengths:
        circle = reverse_section(circle, pos, length)
        pos = (pos + length + skip) % size
        skip += 1
    return circle

In [7]:
def knot_hash(text, rounds=64):
    lengths = (read_lengths(text) + (17, 31, 73, 47, 23)) * rounds
    sparse_hash = tie_knots(256, lengths)
    dense_hash = [reduce(__xor__, sparse_hash[grp:grp+16]) for grp in range(0, 256, 16)]
    return ''.join(f'{char:0{2}x}' for char in dense_hash)

#### Part 1: Calculate occupied squares

In [8]:
@dataclass(frozen=True)
class Point():
    y: int
    x: int
        
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)

In [9]:
def dense_hash_to_digits(dense_hash):
    digits = []
    for char in dense_hash:
        digits.append(int(char, 16))
    return digits

In [10]:
def row(key, row_number):
    digits = dense_hash_to_digits(knot_hash(f'{key}-{row_number}'))
    return ''.join(f'{digit:04b}' for digit in digits)

In [11]:
def occupied_squares(key):
    occupied = set()
    for y in range(128):
        for x, char in enumerate(row(key, y)):
            if char == '1':
                occupied.add(Point(y, x))
    return occupied

In [12]:
occupied = occupied_squares(data)
p1 = len(occupied)
print(f'Part 1: {p1}')

Part 1: 8106


#### Part 2: Distinct regions

In [13]:
ADJACENT = [
    Point(0, -1),
    Point(0, 1),
    Point(1, 0),
    Point(-1, 0),
]

In [14]:
def region(occupied, origin, visited=None):
    visited = visited if visited else set()
    visited.add(origin)
    for direction in ADJACENT:
        neighbour = origin + direction
        if (neighbour in occupied) and (neighbour not in visited):
            visited = visited.union(region(occupied, neighbour, visited))
    return visited

In [15]:
def all_regions(occupied):
    regions = set()
    mapped = set()
    while len(mapped) < len(occupied):
        candidate = next(occ for occ in occupied if occ not in mapped)
        candidate_region = region(occupied, candidate)
        mapped = mapped.union(candidate_region)
        regions.add(tuple((pt.y, pt.x) for pt in candidate_region))
    return regions

In [16]:
regions = all_regions(occupied)
p2 = len(regions)
print(f'Part 2: {p2}')

Part 2: 1164
